In [1]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.firefox.options import Options
import time
import os
from fake_useragent import UserAgent
from random import randint
import pandas as pd
import numpy as np
import re
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import requests
from sqlalchemy.exc import IntegrityError
import string

In [2]:
options = Options()
ua = UserAgent()
userAgent = ua.random
print(userAgent)
options.add_argument(f'user-agent={userAgent}')
#options.add_argument("--headless")
driver = webdriver.Firefox(firefox_options=options)
#     driver = webdriver.Firefox()

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/wafistos/anaconda3/lib/python3.8/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/home/wafistos/anaconda3/lib/python3.8/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.2 Safari/537.36


<ipython-input-2-5315509cced7>:7: DeprecationWarning: use options instead of firefox_options
  driver = webdriver.Firefox(firefox_options=options)


# caculate time

In [3]:
driver.get('https://homzmart.sa/product-vendor/2#1')

In [142]:
start = time.perf_counter()
finish = time.perf_counter()

print( f'Finished  in { round(finish-start, 2) }) second(s')

Finished  in -0.0) second(s


# Multiprocessing test with concurrent

In [152]:
import concurrent
# Link tuto YT https://www.youtube.com/watch?v=fKl2JW_qrso

def do_something(seconds):
    print(f'Sleeping {seconds} second..\n')
    time.sleep(seconds)
    return f'Done Sleeping {seconds}...'

# with concurrent.futures.ThreadPoolExecutor() as excutor:
with concurrent.futures.ProcessPoolExecutor() as excutor:
    # Creer des Muliprocessing 
    results = [ excutor.submit( do_something, 1) for _ in range(10)]
    # Creer map Muliprocessing 
#     excutor.map(do_something, List_pour iterer)
    
    # Get result of Multiprocessing
    for f in concurrent.futures.as_completed(results):
        print(f.result())

Sleeping 1 second..
Sleeping 1 second..
Sleeping 1 second..
Sleeping 1 second..
Sleeping 1 second..
Sleeping 1 second..






Sleeping 1 second..
Sleeping 1 second..

Sleeping 1 second..
Sleeping 1 second..



Done Sleeping 1...
Done Sleeping 1...
Done Sleeping 1...
Done Sleeping 1...
Done Sleeping 1...
Done Sleeping 1...
Done Sleeping 1...
Done Sleeping 1...
Done Sleeping 1...
Done Sleeping 1...


In [155]:
driver.get('https://www.alrugaibfurniture.com/ar/collections/candle-holders/products/metal-candle-holder-hurricane-medium')

# Get url product

In [49]:
first_test = False
def get_data(url):
    """
    Fonction to scrape all urls from itch categories
    Return Data
    """
#     s = HTMLSession()
#     r = s.get(url)
#     r.html.render(timeout= 30, sleep=1)
    global first_test
    print('Url:', url)
    if not first_test:
        driver.get(url)
        
        first_test = True
    time.sleep(3) 
    soup1 = driver.find_element_by_css_selector('body').get_attribute('innerHTML')
    soup = BeautifulSoup(soup1, 'html.parser')
    urls = soup.find_all('div', {'class':'product-card'})
#     soup = BeautifulSoup(r.content, "html.parser")
    time.sleep(1)
    products = soup.find_all('li', {'class': 'item last'})
    liens = ['https://homzmart.sa' + url.find('a')['href'] for url in urls]
    print('Len products', len(liens))
    list_liens = []
    
    for t in liens:
        list_liens.append(t)
#     print('list_cat1', list_cat1)
    data = {
        'urls':list_liens,
        }
    # df = pd.DataFrame(data)
#     print(df)
#         print('Href: ', t['href'])
#     print("Soup get_data")
    return list_liens


def getnextpage():
    """
    Check if next url exist else send None objects
    Return URL or None
    """
    time.sleep(5)
    click_btn = driver.find_element_by_css_selector('.v-pagination > li:nth-child(7) > button:nth-child(1) > i:nth-child(1)')
#     page = soup.find('a', {'class': 'next i-next'})
    # print('Page', page)

    click_btn.click()

# Extract new urls of Rugaib site from url categories

# url = 'https://rawae.com/search?subsubcategory=Accessories-box-Q3HaJ&page=1'
list_urls = []

data = []
def scrap_url_product(url, df):
    global first_test 
#     cat1 = url1['cat1']
#     cat2 = url1['cat2']
#     cat3 = url1['cat3']
#     url = url1['url']
#     data = []
#     print(cat1, cat2, cat3)

    while True:
        urls_list = get_data('https://homzmart.sa/products/'+ url['url'])
        
        
        for toto in urls_list:
            # print(f'URL:', toto)
            data.append({
            'url':toto,
            'cat1': url['cat1'],
            'cat2': url['cat2'],
            'cat3': url['cat3'],
            })
       

        try:
            getnextpage()
#             print('Url dans le while', url)
        except:
            print('pas Next')
            first_test = False
            break
    # print(data)
    return data
    print( f'Scrape done .')

In [52]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/homzmart/home_model.xlsx')

for tt in list_url:
    data = scrap_url_product(tt, df)
    df1 = pd.DataFrame(data)
    df = pd.concat([df, df1], ignore_index=True)
    df.to_excel('homzmart_urls.xlsx')

Url: https://homzmart.sa/products/229#1
Len products 20
Url: https://homzmart.sa/products/229#1
Len products 20
Url: https://homzmart.sa/products/229#1
Len products 0
Url: https://homzmart.sa/products/229#1
Len products 0
pas Next
Url: https://homzmart.sa/products/233#1
Len products 12
pas Next
Url: https://homzmart.sa/products/234#1
Len products 20
pas Next
Url: https://homzmart.sa/products/235#1
Len products 20
Url: https://homzmart.sa/products/235#1
Len products 20
Url: https://homzmart.sa/products/235#1
Len products 20
Url: https://homzmart.sa/products/235#1
Len products 20
Url: https://homzmart.sa/products/235#1
Len products 15
pas Next
Url: https://homzmart.sa/products/237#1
Len products 20
pas Next
Url: https://homzmart.sa/products/240#1
Len products 3
pas Next
Url: https://homzmart.sa/products/247#1
Len products 20
pas Next
Url: https://homzmart.sa/products/266#1
Len products 0
pas Next
Url: https://homzmart.sa/products/105#1
Len products 0
pas Next
Url: https://homzmart.sa/pro

In [46]:
list_url  = [{ 'url': '229#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'الأجهزة الكهربائية الصغيرة', 'cat3': ''}
,{ 'url': '233#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'مبردات وموزعات المياه', 'cat3': ''}
,{ 'url': '234#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'غسالات ومجففات', 'cat3': ''}
,{ 'url': '235#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'أنظمة التبريد والتدفئة', 'cat3': ''}
,{ 'url': '237#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'ثلاجات ومجمدات', 'cat3': ''}
,{ 'url': '240#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'غسالات الصحون', 'cat3': ''}
,{ 'url': '247#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'مواقد الطهي و الأفران', 'cat3': ''}
,{ 'url': '266#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'التلفزيونات', 'cat3': ''}


,{ 'url': '105#1', 'cat1': 'المطبخ', 'cat2': 'أدوات الطعام', 'cat3': ''}
,{ 'url': '104#1', 'cat1': 'المطبخ', 'cat2': 'أدوات الشرب', 'cat3': ''}
,{ 'url': '103#1', 'cat1': 'المطبخ', 'cat2': 'أدوات الطبخ', 'cat3': ''}
,{ 'url': '90#1', 'cat1': 'المطبخ', 'cat2': 'صوانى', 'cat3': ''}
,{ 'url': '87#1', 'cat1': 'المطبخ', 'cat2': 'أدوات التقديم', 'cat3': ''}



,{ 'url': '70#1', 'cat1': 'الديكور', 'cat2': 'الإضاءة', 'cat3': ''}
,{ 'url': '78#1', 'cat1': 'الديكور', 'cat2': 'إكسسوار الديكور', 'cat3': ''}
,{ 'url': '65#1', 'cat1': 'الديكور', 'cat2': 'سجاد', 'cat3': ''}
,{ 'url': '91#1', 'cat1': 'الديكور', 'cat2': 'المفروشات', 'cat3': ''}
,{ 'url': '59#1', 'cat1': 'الديكور', 'cat2': 'الستائر و المفروشات', 'cat3': ''}
,{ 'url': '409#1', 'cat1': 'الديكور', 'cat2': 'الديكور المنزلي', 'cat3': ''}
,{ 'url': '328#1', 'cat1': 'الديكور', 'cat2': 'مجموعة رمضان', 'cat3': ''}
,{ 'url': '408#1', 'cat1': 'الديكور', 'cat2': 'هدايا', 'cat3': ''}



,{ 'url': '428#1', 'cat1': 'الرياضة والألعاب', 'cat2': 'كرسي الألعاب', 'cat3': ''}
,{ 'url': '426#1', 'cat1': 'الرياضة والألعاب', 'cat2': 'أداوات الرياضة المنزلية', 'cat3': ''}
,{ 'url': '429#1', 'cat1': 'الرياضة والألعاب', 'cat2': 'طاولات الألعاب', 'cat3': ''}
,{ 'url': '383#1', 'cat1': 'الرياضة والألعاب', 'cat2': 'ألعاب اللوح', 'cat3': ''}
,{ 'url': '7#1', 'cat1': 'الرياضة والألعاب', 'cat2': 'ألعاب', 'cat3': ''}

,{ 'url': '28#1', 'cat1': 'وحدات التخزين', 'cat2': 'وحدة تلفزيون', 'cat3': ''}
,{ 'url': '30#1', 'cat1': 'وحدات التخزين', 'cat2': 'وحدة أدراج', 'cat3': ''}
,{ 'url': '40#1', 'cat1': 'وحدات التخزين', 'cat2': 'ركن قهوة', 'cat3': ''}
,{ 'url': '37#1', 'cat1': 'وحدات التخزين', 'cat2': 'خزانة', 'cat3': ''}
,{ 'url': '31#1', 'cat1': 'وحدات التخزين', 'cat2': 'بوفيه', 'cat3': ''}
,{ 'url': '32#1', 'cat1': 'وحدات التخزين', 'cat2': 'وحدة عرض', 'cat3': ''}
,{ 'url': '39#1', 'cat1': 'وحدات التخزين', 'cat2': 'مكتبات', 'cat3': ''}
,{ 'url': '33#1', 'cat1': 'وحدات التخزين', 'cat2': 'رفوف تخزين الأحذية', 'cat3': ''}
,{ 'url': '34#1', 'cat1': 'وحدات التخزين', 'cat2': 'وحدة تخزين حمام', 'cat3': ''}
,{ 'url': '35#1', 'cat1': 'وحدات التخزين', 'cat2': 'وحده تخزين مطبخ', 'cat3': ''}
,{ 'url': '164#1', 'cat1': 'وحدات التخزين', 'cat2': 'علاقّات الملابس', 'cat3': ''}

,{ 'url': '8#1', 'cat1': 'الاثاث', 'cat2': 'غرف نوم', 'cat3': 'أسرّة'}
,{ 'url': '38#1', 'cat1': 'الاثاث', 'cat2': 'غرف نوم', 'cat3': 'طاولات التسريحة'}
,{ 'url': '41#1', 'cat1': 'الاثاث', 'cat2': 'غرف نوم', 'cat3': 'كومودينه'}
,{ 'url': '92#1', 'cat1': 'الاثاث', 'cat2': 'غرف نوم', 'cat3': 'مراتب'}
,{ 'url': '396#1', 'cat1': 'الاثاث', 'cat2': 'غرف نوم', 'cat3': 'أطقم غرف نوم'}
,{ 'url': '10#1', 'cat1': 'الاثاث', 'cat2': 'غرفة المعيشة', 'cat3': 'كرسي'}
,{ 'url': '18#1', 'cat1': 'الاثاث', 'cat2': 'غرفة المعيشة', 'cat3': 'كنبة'}
,{ 'url': '23#1', 'cat1': 'الاثاث', 'cat2': 'غرفة المعيشة', 'cat3': 'طاولة'}
,{ 'url': '378#1', 'cat1': 'الاثاث', 'cat2': 'غرفة المعيشة', 'cat3': 'اطقم غرف المعيشة'}
,{ 'url': '379#1', 'cat1': 'الاثاث', 'cat2': 'غرفة المعيشة', 'cat3': 'أطقم غرف التلفزيون'}
,{ 'url': '47#1', 'cat1': 'الاثاث', 'cat2': 'أثاث خارجي', 'cat3': 'كراسى معلقة'}
,{ 'url': '49#1', 'cat1': 'الاثاث', 'cat2': 'أثاث خارجي', 'cat3': 'مقعد محشو'}
,{ 'url': '50#1', 'cat1': 'الاثاث', 'cat2': 'أثاث خارجي', 'cat3': 'كنبة أثاث خارجى'}
,{ 'url': '51#1', 'cat1': 'الاثاث', 'cat2': 'أثاث خارجي', 'cat3': 'شوايات'}
,{ 'url': '185#1', 'cat1': 'الاثاث', 'cat2': 'أثاث خارجي', 'cat3': 'كرسي خارجي'}
,{ 'url': '186#1', 'cat1': 'الاثاث', 'cat2': 'أثاث خارجي', 'cat3': 'طاولة خارجية'}
,{ 'url': '56#1', 'cat1': 'الاثاث', 'cat2': 'المكتب', 'cat3': 'كرسى مكتب'}
,{ 'url': '57#1', 'cat1': 'الاثاث', 'cat2': 'المكتب', 'cat3': 'مكتب'}
,{ 'url': '11#1', 'cat1': 'الاثاث', 'cat2': 'غرف الطعام', 'cat3': 'كرسي غرفة الطعام'}
,{ 'url': '24#1', 'cat1': 'الاثاث', 'cat2': 'غرف الطعام', 'cat3': 'طاولة غرفة الطعام'}

         ]








# Strat scrape urls products

In [13]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/homzmart/home_model.xlsx')
scrap_url_product(list_url, df)


Url: [{'url': '105#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'الأجهزة الكهربائية الصغيرة', 'cat3': ''}, {'url': '105#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'مبردات وموزعات المياه', 'cat3': ''}, {'url': '105#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'غسالات ومجففات', 'cat3': ''}, {'url': '105#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'أنظمة التبريد والتدفئة', 'cat3': ''}, {'url': '105#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'ثلاجات ومجمدات', 'cat3': ''}, {'url': '105#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'غسالات الصحون', 'cat3': ''}, {'url': '105#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'مواقد الطهي و الأفران', 'cat3': ''}, {'url': '105#1', 'cat1': 'الاجهزة المنزلية', 'cat2': 'التلفزيونات', 'cat3': ''}, {'url': '105#1', 'cat1': 'المطبخ', 'cat2': 'أدوات الطعام', 'cat3': ''}, {'url': '104#1', 'cat1': 'المطبخ', 'cat2': 'أدوات الشرب', 'cat3': ''}, {'url': '103#1', 'cat1': 'المطبخ', 'cat2': 'أدوات الطبخ', 'cat3': ''}, {'url': '90#1', 'cat1': 'المطبخ', 'cat2': 'صوانى', 'cat3': ''}, {'url': '87#1'

NameError: name 'r' is not defined

In [10]:
driver.find_

### Create elements

In [136]:

def fresh():
    soup1 = driver.find_element_by_css_selector('body').get_attribute('innerHTML')
    soup = BeautifulSoup(soup1 , 'html.parser')
    return soup
soup = fresh()

In [63]:

list_elements = soup.find_all('ul',{'data-v-26f8ee82': ''})
len(list_elements)

8

In [95]:
toto = soup.find('h4', text= re.compile(".*SKU.*")).next_element.next_element.replace(':', '').strip()
toto

'AR.14585-03'

In [80]:
for tt in list_elements:
    try:
        print(tt.find('h4').text)
        break
    except:
        pass


                                التوصيل خلال
                            


In [103]:
sku = extraire_ele('SKU', soup)
sku

'AR.14585-03'

In [104]:
size = extraire_ele('المقاسات', soup)
size

'23x23x18 cm'

In [105]:
raw_materials = extraire_ele('الخامات', soup)
raw_materials

'ألومنيوم'

In [173]:
manufacturer = extraire_details('بلد المنشأ', soup)
manufacturer

'مصر'

In [108]:
delevery_days = extraire_ele('التوصيل خلال', soup)
delevery_days

'7-10 أيام عمل'

In [109]:
is_is_stock = extraire_ele('المنتج متاح', soup)
is_is_stock

'On Demand'

In [126]:
description = soup.find('', text= re.compile(f".*وصف المنتج.*")).next_element.next_element.text
description

'العلامة التجارية: سيجبروكالطول (سم): 23العرض (سم): 23الإرتفاع (سم): 18اللون: أسودالخامة: ألومنيومبلد المنشأ: المملكة العربية السعودية'

In [178]:
barcode = extraire_details('رقم الموديل', soup)
type_ = extraire_details('نوع', soup)
barcode

'1803-OT6-QS'

In [180]:
type_ = extraire_details('نوع', soup)
type_

'سجاد ارضيات'

In [131]:
def extraire_details(name, list_details):
    try:
        toto = list_details.find('li', text= re.compile(f".*{name}.*")).text.replace(':', '').replace(name, '').strip()
    except:
        toto = ''
    return toto

In [127]:
brand_b = 

In [133]:
brand = extraire_details('العلامة التجارية', soup)
brand

'سيجبروك'

In [135]:
free_colors = extraire_details('اللون', soup)
free_colors

'أسود'

In [137]:
name = soup.find('h3', {'class': 'name'}).text.strip()
name

'حامل شموع 30×100 سم من في أي ماركت VI.AZ7052 - ذهبي'

In [139]:
price = soup.find('h3', {'class': 'price'}).text.replace('ر.س', '').strip()
price

'300'

In [161]:
images_list_btn = driver.find_elements_by_xpath('//div[@class="v-responsive__sizer"]')
len(images_list_btn)
list_images = []
for img in images_list_btn:
    img.click()
    soup = fresh()
    images = soup.find('div', {'class': 'flex mini-image-view sm12 md7'}).find('img')
    list_images.append('https://homzmart.sa'+images['src'])

https://homzmart.sa/s3/catalog/product/a/z/az7052-en.jpg
https://homzmart.sa/s3/catalog/product/a/z/az7052-w.jpg
https://homzmart.sa/s3/catalog/product/a/z/az7052-d.jpg


In [146]:
soup = fresh()
images = soup.find('div', {'class': 'flex mini-image-view sm12 md7'}).find('img')
print('https://homzmart.sa'+images['src'])

https://homzmart.sa/s3/catalog/product/a/z/az7052-d.jpg


In [4]:
lsi = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/homzmart/homzmart_urls.xlsx')
test_list = []
for index, row in lsi.iterrows():
    test_list.append(
    {
        'url': row['url'],
        'cat1': row['cat1'],
        'cat2': row['cat2'],
        'cat3': row['cat3'],
    })

# Debut de la fonction

In [6]:
urls_data = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/homzmart/Home_urls.xlsx') 
list_urls = urls_data['url'].to_list()

def extraire_details(name, list_details):
    try:
        toto = list_details.find('li', text= re.compile(f".*{name}.*")).text.replace(':', '').replace(name, '').strip()
    except:
        toto = ''
    return toto

def extraire_ele(name, list_details):
    try:
        toto = list_details.find('h4', text= re.compile(f".*{name}.*")).next_element.next_element.replace(':', '').strip()

    except:
        toto = ''
    return toto

def fresh():
    for i in range(10):
        soup1 = driver.find_element_by_css_selector('body').get_attribute('innerHTML')
        soup = BeautifulSoup(soup1 , 'html.parser')
    return soup

now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
count = 0
list_images = [] 

display_bool = False


#**********************************************************************
#**********************************************************************
#**********************************************************************
#**********************************************************************
#**********************************************************************
#ForLoop main
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/homzmart/home_product_model.xlsx')
for i, toto in enumerate(test_list[16260: ]):

    print('Count:', i)
    print('URL:', toto['url'])
    cat1 = toto['cat1']
    cat2 = toto['cat2']
    cat3 = toto['cat3']
    url = toto['url']
    driver.get(url)
    time.sleep(3)
    soup = fresh()
    toto = driver.find_elements_by_class_name('flex')
    test = False
    for i in toto:
        if 'الرقيب للاثاث' in i.text:
            print('Rugaib trouve')
            test = True
            break
    if not test:
        print('pas rugaib')
        continue
    sku = extraire_ele('SKU', soup)
    try:
        
        name = driver.find_element_by_css_selector('.name').text
    except:
        time.sleep(3)
        name = driver.find_element_by_css_selector('.name').text
        soup = fresh()
    soup = fresh()
    brand = extraire_details('العلامة التجارية', soup)
    soup = fresh()
    price = soup.find('h3', {'class': 'price'}).text.replace('ر.س', '').strip()
    soup = fresh()
    description = soup.find('', text= re.compile(f".*وصف المنتج.*")).next_element.next_element.text
    time.sleep(1)
    soup = fresh()
    images_list_btn = driver.find_elements_by_xpath('//div[@class="v-responsive__sizer"]')
    len(images_list_btn)
    soup = fresh()
    list_images = []
    for img in images_list_btn:
        img.click()
        soup = fresh()
        images = soup.find('div', {'class': 'flex mini-image-view sm12 md7'}).find('img')
        list_images.append('https://homzmart.sa'+images['src'])
    is_in_stock = extraire_ele('المنتج متاح', soup)
    soup = fresh()
    mgs_brand = extraire_details('العلامة التجارية', soup)
    soup = fresh()
    dim = extraire_ele('المقاسات', soup)
    soup = fresh()
    raw_material = extraire_ele('الخامات', soup)
    soup = fresh()
    free_colors = extraire_details('اللون', soup)
    delevery_days = extraire_ele('التوصيل خلال', soup)
    manufacturer = extraire_details('بلد المنشأ', soup)
    barcode = extraire_details('رقم الموديل', soup)
    type_ = extraire_details('نوع', soup)
#************************************************************************** 
#     print('URL:', url)
#     print('Price:', price)




#         list_input_images2.append(t)
#*******************************************************
    two_month = datetime.now() + timedelta(days=60)
    two_month = two_month.strftime("%m/%d/%Y")
    today = datetime.today().strftime("%m/%d/%Y")
    try:
        base_image = list_images[0]
    except:
        continue
    additionnal_images = ','.join(list_images[1:])
#*******************************************************

#     url_key = f'{sku} {title}'

#     url_key = url_key.replace(',', '-').replace('*', '-').replace('،', '-')


    series = pd.Series([sku,
                         name,
                         price,
                         free_colors,#colors,
                        raw_material, 
                        description,
                        barcode,
                        type_,
                         dim,#width,
                         mgs_brand,
                         base_image,
                        additionnal_images,
                         today,
                         two_month,
                         url,#link_url,
                        manufacturer,
                        delevery_days,
                        is_in_stock,
                        cat1,
                        cat2,
                        cat3,

                        ],  index=df.columns)
    #df.append(df1)
    df = df.append(series, ignore_index=True)
    df.to_excel( f'homzmart_test3.xlsx', index = False)


Count: 0
URL: https://homzmart.sa/product-details/SHTV27-70
pas rugaib
Count: 1
URL: https://homzmart.sa/product-details/DA.BU-19
pas rugaib
Count: 2
URL: https://homzmart.sa/product-details/SHS.204545-3
pas rugaib
Count: 3
URL: https://homzmart.sa/product-details/MA.NHS055A
pas rugaib
Count: 4
URL: https://homzmart.sa/product-details/ALR.W215-66
Rugaib trouve
Count: 5
URL: https://homzmart.sa/product-details/SHTV17-79
pas rugaib
Count: 6
URL: https://homzmart.sa/product-details/GF-849-C
Rugaib trouve
Count: 7
URL: https://homzmart.sa/product-details/ALR.B719-46
Rugaib trouve
Count: 8
URL: https://homzmart.sa/product-details/AM-02-23013-H66-00-34
pas rugaib
Count: 9
URL: https://homzmart.sa/product-details/ALR.B267-46
Rugaib trouve
Count: 10
URL: https://homzmart.sa/product-details/AM-02-23013-B39-01-35
pas rugaib
Count: 11
URL: https://homzmart.sa/product-details/AM-02-23013-H2T-01-30
pas rugaib
Count: 12
URL: https://homzmart.sa/product-details/ALR.b672-46
Rugaib trouve
Count: 13
URL

pas rugaib
Count: 110
URL: https://homzmart.sa/product-details/SUM.OMF150
pas rugaib
Count: 111
URL: https://homzmart.sa/product-details/K.RDNE18C2E21S
pas rugaib
Count: 112
URL: https://homzmart.sa/product-details/K.Yorx0d78-1
pas rugaib
Count: 113
URL: https://homzmart.sa/product-details/K.HS-121L(NK)
pas rugaib
Count: 114
URL: https://homzmart.sa/product-details/K.R-V600PS7KTWH
pas rugaib
Count: 115
URL: https://homzmart.sa/product-details/K.RDNE15C9E21S
pas rugaib
Count: 116
URL: https://homzmart.sa/product-details/SUM.OMF300
pas rugaib
Count: 117
URL: https://homzmart.sa/product-details/K.RT35K5157SLC
pas rugaib
Count: 118
URL: https://homzmart.sa/product-details/K.FUSQ603
pas rugaib
Count: 119
URL: https://homzmart.sa/product-details/K.FGC5555SF
pas rugaib
Count: 120
URL: https://homzmart.sa/product-details/K.AL-H32EX
pas rugaib
Count: 121
URL: https://homzmart.sa/product-details/K.AL-OK94GX
pas rugaib
Count: 122
URL: https://homzmart.sa/product-details/K.NT6500SRC1
pas rugaib
Co

pas rugaib
Count: 220
URL: https://homzmart.sa/product-details/K.MFK100
pas rugaib
Count: 221
URL: https://homzmart.sa/product-details/SUM.OMR50
pas rugaib
Count: 222
URL: https://homzmart.sa/product-details/SUM.ROW4854DXHZ-19
pas rugaib
Count: 223
URL: https://homzmart.sa/product-details/K.MSTE12HRN2AB4
pas rugaib
Count: 224
URL: https://homzmart.sa/product-details/K.DWC-ELD1432
pas rugaib
Count: 225
URL: https://homzmart.sa/product-details/MA.NB042
pas rugaib
Count: 226
URL: https://homzmart.sa/product-details/K.NWM700FN21
pas rugaib
Count: 227
URL: https://homzmart.sa/product-details/SUM.RO1294DXH5Z-19
pas rugaib
Count: 228
URL: https://homzmart.sa/product-details/K.MDG80CS
pas rugaib
Count: 229
URL: https://homzmart.sa/product-details/K.UWMFL10
pas rugaib
Count: 230
URL: https://homzmart.sa/product-details/K.BD-70CVE-AWH
pas rugaib
Count: 231
URL: https://homzmart.sa/product-details/K.MSTE30CRN2AB4
pas rugaib
Count: 232
URL: https://homzmart.sa/product-details/K.WWFLD10VW08
pas rug

pas rugaib
Count: 329
URL: https://homzmart.sa/product-details/K.GWC-18AGD-D3NTA1A
pas rugaib
Count: 330
URL: https://homzmart.sa/product-details/K.NT382C
pas rugaib
Count: 331
URL: https://homzmart.sa/product-details/K.ATQ24GPLT0-T2
pas rugaib
Count: 332
URL: https://homzmart.sa/product-details/K.Yorx10
pas rugaib
Count: 333
URL: https://homzmart.sa/product-details/K.Yorx03
pas rugaib
Count: 334
URL: https://homzmart.sa/product-details/K.UAWG24H
pas rugaib
Count: 335
URL: https://homzmart.sa/product-details/K.Yorxfreezer-1
pas rugaib
Count: 336
URL: https://homzmart.sa/product-details/K.HD-606FSEN
pas rugaib
Count: 337
URL: https://homzmart.sa/product-details/K.RT35K5157WWC
pas rugaib
Count: 338
URL: https://homzmart.sa/product-details/K.Yorxfreezer
pas rugaib
Count: 339
URL: https://homzmart.sa/product-details/K.C205-HC
pas rugaib
Count: 340
URL: https://homzmart.sa/product-details/K.HRC71S
pas rugaib
Count: 341
URL: https://homzmart.sa/product-details/K.RDNE15C9E21W
pas rugaib
Count

pas rugaib
Count: 440
URL: https://homzmart.sa/product-details/AHR.F-9
pas rugaib
Count: 441
URL: https://homzmart.sa/product-details/AHR.F-13
pas rugaib
Count: 442
URL: https://homzmart.sa/product-details/K.311115040
pas rugaib
Count: 443
URL: https://homzmart.sa/product-details/K.311115048
pas rugaib
Count: 444
URL: https://homzmart.sa/product-details/AHR.F-585
pas rugaib
Count: 445
URL: https://homzmart.sa/product-details/AHR.F-328
pas rugaib
Count: 446
URL: https://homzmart.sa/product-details/AHR.F-54
pas rugaib
Count: 447
URL: https://homzmart.sa/product-details/AHR.F-339
pas rugaib
Count: 448
URL: https://homzmart.sa/product-details/AHR.F-11
pas rugaib
Count: 449
URL: https://homzmart.sa/product-details/AHR.F-638
pas rugaib
Count: 450
URL: https://homzmart.sa/product-details/AHR.F-140
pas rugaib
Count: 451
URL: https://homzmart.sa/product-details/AHR.F-109
pas rugaib
Count: 452
URL: https://homzmart.sa/product-details/AHR.F-248
pas rugaib
Count: 453
URL: https://homzmart.sa/produ

...

pas rugaib
Count: 20933
URL: https://homzmart.sa/product-details/OV.1040490-3
pas rugaib
Count: 20934
URL: https://homzmart.sa/product-details/OV.1040626-5
pas rugaib
Count: 20935
URL: https://homzmart.sa/product-details/OV.1040626-6
pas rugaib
Count: 20936
URL: https://homzmart.sa/product-details/OV.1040490-5
pas rugaib
Count: 20937
URL: https://homzmart.sa/product-details/MA.NHFS-T1838W
pas rugaib
Count: 20938
URL: https://homzmart.sa/product-details/MA.NHLIN072
pas rugaib
Count: 20939
URL: https://homzmart.sa/product-details/OV.1040491-5
pas rugaib
Count: 20940
URL: https://homzmart.sa/product-details/AR.15646-02
Rugaib trouve
Count: 20941
URL: https://homzmart.sa/product-details/AR.15646-01
Rugaib trouve
Count: 20942
URL: https://homzmart.sa/product-details/RGLINHSE.021110500027
pas rugaib
Count: 20943
URL: https://homzmart.sa/product-details/RGLINHSE.021110500806
pas rugaib
Count: 20944
URL: https://homzmart.sa/product-details/RGLINHSE.021110500020
pas rugaib
Count: 20945
URL: htt

pas rugaib
Count: 21039
URL: https://homzmart.sa/product-details/ALR.B733-45
Rugaib trouve
Count: 21040
URL: https://homzmart.sa/product-details/AM-02-23013-H0D-01-30
pas rugaib
Count: 21041
URL: https://homzmart.sa/product-details/VI.40-18843
pas rugaib
Count: 21042
URL: https://homzmart.sa/product-details/ALR.B633-46
Rugaib trouve
Count: 21043
URL: https://homzmart.sa/product-details/ALR.B497-46
Rugaib trouve
Count: 21044
URL: https://homzmart.sa/product-details/GF-849-C
Rugaib trouve
Count: 21045
URL: https://homzmart.sa/product-details/ALR.B719-46
Rugaib trouve
Count: 21046
URL: https://homzmart.sa/product-details/AM-02-23013-H66-00-34
pas rugaib
Count: 21047
URL: https://homzmart.sa/product-details/ALR.B267-46
Rugaib trouve
Count: 21048
URL: https://homzmart.sa/product-details/AM-02-23013-B39-01-35
pas rugaib
Count: 21049
URL: https://homzmart.sa/product-details/AM-02-23013-H2T-01-30
pas rugaib
Count: 21050
URL: https://homzmart.sa/product-details/ALR.b672-46
Rugaib trouve
Count: 

Rugaib trouve
Count: 21142
URL: https://homzmart.sa/product-details/ALR.B749-31
Rugaib trouve
Count: 21143
URL: https://homzmart.sa/product-details/ALR.B560-31
Rugaib trouve
Count: 21144
URL: https://homzmart.sa/product-details/ALR.B697-31
Rugaib trouve
Count: 21145
URL: https://homzmart.sa/product-details/ALR.B594-31
Rugaib trouve
Count: 21146
URL: https://homzmart.sa/product-details/ALR.B733-31
Rugaib trouve
Count: 21147
URL: https://homzmart.sa/product-details/ALR.B070-31
Rugaib trouve
Count: 21148
URL: https://homzmart.sa/product-details/ALR.B446-46
Rugaib trouve
Count: 21149
URL: https://homzmart.sa/product-details/ALR.B733-91
Rugaib trouve
Count: 21150
URL: https://homzmart.sa/product-details/ALR.B763-93
Rugaib trouve
Count: 21151
URL: https://homzmart.sa/product-details/ALR.B744-92
Rugaib trouve
Count: 21152
URL: https://homzmart.sa/product-details/ALR.B647-191
Rugaib trouve
Count: 21153
URL: https://homzmart.sa/product-details/ALR.B181-92
Rugaib trouve
Count: 21154
URL: https:/

pas rugaib
Count: 21249
URL: https://homzmart.sa/product-details/CA.A-10
pas rugaib
Count: 21250
URL: https://homzmart.sa/product-details/CA.A-9
pas rugaib
Count: 21251
URL: https://homzmart.sa/product-details/CA.A-15
pas rugaib
Count: 21252
URL: https://homzmart.sa/product-details/CA.A-11
pas rugaib
Count: 21253
URL: https://homzmart.sa/product-details/CA.A-36
pas rugaib
Count: 21254
URL: https://homzmart.sa/product-details/CA.A-5
pas rugaib
Count: 21255
URL: https://homzmart.sa/product-details/CA.A-50
pas rugaib
Count: 21256
URL: https://homzmart.sa/product-details/CA.A-54
pas rugaib
Count: 21257
URL: https://homzmart.sa/product-details/CA.A-37
pas rugaib
Count: 21258
URL: https://homzmart.sa/product-details/CA.A-47
pas rugaib
Count: 21259
URL: https://homzmart.sa/product-details/CA.A-43
pas rugaib
Count: 21260
URL: https://homzmart.sa/product-details/CA.A-4
pas rugaib
Count: 21261
URL: https://homzmart.sa/product-details/CA.A-13
pas rugaib
Count: 21262
URL: https://homzmart.sa/produ

NoSuchElementException: Message: Unable to locate element: .name


In [15]:
df.shape

(91, 21)

In [12]:
df.columns

Index(['sku', 'name', 'price', 'free_colors', 'raw_material', 'description',
       'barcode', 'type_', 'dimension', 'mgs_brand', 'base_image',
       'additionnnal_images', 'tody', 'two_month', 'url', 'manufacturer',
       'delevry_days', 'is_in_stock', 'cat1', 'cat2', 'cat3'],
      dtype='object')

In [8]:
df1 =pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/homzmart/homzmart_test.xlsx')
df2 =pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/homzmart/homzmart_test2.xlsx')


In [10]:
df = pd.concat([df, df1])

In [14]:
df.drop_duplicates(subset=['sku'], inplace=True)

In [16]:
name = driver.find_element_by_css_selector('.name').text
name

'كرسي 64×86×56 سم من توف TOV-D47 - رمادي'

In [70]:
toto = driver.find_elements_by_class_name('flex')


In [72]:
toto = driver.find_elements_by_class_name('flex')
for i in toto:

    if 'الرقيب للاثاث' in i.text:
        print(i.text)


مصباح طاولة 46×96×25 سم من الرقيب للاثاث ALR.26444-1-UTT - متعدد الألوان
1,429 ر.س
أضف الى السلة
تحتاج للمساعدة
scatter_plot
معلومات المنتج
يباع بواسطة : الرقيب للاثاث
التوصيل خلال : 7-10 أيام عمل
المقاسات : 46×96×25 cm
المنتج متاح : In Stock
الخامات : سيراميك ، كتان
صنع فى : SA
SKU : ALR.26444-1-UTT
scatter_plot
معلومات المنتج
يباع بواسطة : الرقيب للاثاث
التوصيل خلال : 7-10 أيام عمل
المقاسات : 46×96×25 cm
المنتج متاح : In Stock
الخامات : سيراميك ، كتان
صنع فى : SA
SKU : ALR.26444-1-UTT
list
وصف المنتج
العلامة التجارية: الرقيب للاثاث
اللون: متعدد الألوان
الخامة: سيراميك، كتان
الطول (سم): 46
الارتفاع (سم): 96
العرض (سم): 25
بلد المنشأ: المملكة العربية السعودية


In [53]:
 extraire_ele('يباع بواسطة', soup)

NameError: name 'extraire_ele' is not defined

In [23]:

name = soup.find('h3', {'class': 'name'}).text.strip()
name

'كرسي 65×84×60 سم من توف TOV-D6187 - زهري'

In [22]:
name = soup.find('h3', {'class': 'name'}).text.strip()

# Fin de la fonction

In [ ]:
len(links_products)

In [26]:
driver.get('https://www.alrugaibfurniture.com/ar/collections/accessories')

In [31]:
price = driver.find_elements_by_class_name('money')
len(price)

40

In [41]:
pric = driver.find_elements_by_xpath('//div[@class="price-list"]//span')
len(pric)
if len(pric)==4:
    price = pric[2].text.replace('SR', '').replace('SAR', '').strip()
    print('price:', price)
    special_price = pric[0].text.replace('SR', '').replace('SAR', '').strip()
    print('special_price:', special_price)
elif len(pric) == 2:
    price = pric[0].text.replace('SR', '').replace('SAR', '').strip()
    print('price:', price)
    special_price = ''
    print('special_price:', special_price)

price: 330
special_price: 


In [37]:
for t in pric:
    print(t.text)

SR 4,495
SR 4,495
SR 6,097
SR 6,097


In [38]:
if len(pric)==4:
    price

toto


# scrape images

In [5]:
list_scrape1 = df['link_url'].to_list()

In [4]:
df =  pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/scrapingalrugaibfurniture/sub_categories/Rugaid_update_06_08_2021v2.xlsx')

In [5]:
list_scrape1 = df['link_url'].to_list()

In [16]:
df_images1 = pd.read_excel('model_images1.xlsx')
toto_images = []

def scrape_images():
    list_images = []
    images_click = driver.find_elements_by_class_name('product-gallery__thumbnail')


    for ele in images_click:
        #ele.click()
        driver.execute_script("arguments[0].click();", ele)
        time.sleep(0.5)
    images = driver.find_elements_by_class_name('product-gallery__image')
    len(images)
    for img in images:
        #list_images.append('https:'+img.get_attribute('data-srcset').split(' ')[0])
    #     print(':', img.get_attribute('srcset').split(', ')[0].split('?')[0])
        time.sleep(1)
        try:
            t = 'https:'+img.get_attribute('data-srcset').split(', ')[0].split('?')[0]
        except:
            continue
    #     print(img.get_attribute('data-srcset'))
    #     soup = BeautifulSoup(img.get_attribute('innerHTML'), 'html.parser')
    #     print('toto',soup)
        print(t)
        list_images.append(t)
    print('len list images',len(list_images))
    try:
        timage = ','.join(list_images[1:])
    #         list_input_images2.append(timage)
        print(timage)
    except:
        timage = ''
    print('images2-', timage)
    image1 = list_images[0]
    image2 = timage
    return image1, image2
for i, toto in enumerate(list_scrape1[741:]):
    print('*****************************************')
    print('Count', i)
    url = ''

    driver.get(toto)
    
    time.sleep(2)
    try:
        image1, image2 = scrape_images()
    except:
        driver.get(toto)
        image1, image2 = scrape_images()
        
    
    url = toto
    series = pd.Series([url, image1, image2],  index=df_images1.columns)
    
    df_images1 = df_images1.append(series, ignore_index=True)
    df_images1.to_excel( f'update_rugaid_images4.xlsx', index = False)

*****************************************
Count 0
len list images 0

images2- 
len list images 0

images2- 


IndexError: list index out of range

In [20]:
df_images1

,url,images1,image2


In [11]:
list_images[0]

IndexError: list index out of range

In [13]:
def scrape_images():
    list_images = []
    images_click = driver.find_elements_by_class_name('product-gallery__thumbnail')


    for ele in images_click:
        #ele.click()
        driver.execute_script("arguments[0].click();", ele)
        time.sleep(0.5)
    images = driver.find_elements_by_class_name('product-gallery__image')
    len(images)
    for img in images:
        #list_images.append('https:'+img.get_attribute('data-srcset').split(' ')[0])
    #     print(':', img.get_attribute('srcset').split(', ')[0].split('?')[0])
        time.sleep(1)
        try:
            t = 'https:'+img.get_attribute('data-srcset').split(', ')[0].split('?')[0]
        except:
            continue
    #     print(img.get_attribute('data-srcset'))
    #     soup = BeautifulSoup(img.get_attribute('innerHTML'), 'html.parser')
    #     print('toto',soup)
        print(t)
        list_images.append(t)
    print('len list images',len(list_images))
    try:
        timage = ','.join(list_images[1:])
    #         list_input_images2.append(timage)
        print(timage)
    except:
        timage = ''
    print('images2-', timage)
    image1 = list_images[0]
    image2 = timage
    return image1, image2

https://cdn.shopify.com/s/files/1/0773/5495/products/ciiktkasisyg3bumavxb_400x.jpg
len list images 1

images2- 


('https://cdn.shopify.com/s/files/1/0773/5495/products/ciiktkasisyg3bumavxb_400x.jpg',
 '')

In [64]:
def regex_pass(toto):
    
#     if cas2_regex.search(toto) != None:
#         print('regex2')
#     if cas3_regex.search(toto) != None:
#         print('regex3')
#     if cas4_regex.search(toto) != None:
#         print('regex4')
    if cas5_regex.search(toto) != None:
        print('regex5')
        return  filter_data_size_regex5(toto)
#         print(toto)
            
    elif cas1_regex.search(toto) != None:
        print('regex1')
        return  filter_data_size_regex1(toto)
        
    elif cas7_regex.search(toto) != None:
        print('regex7')
        return filter_data_size_regex7(toto)
    
    elif cas21_regex.search(toto) != None:
        print('regex21')
        return filter_data_size_regex21(toto)
        
    elif cas8_regex.search(toto) != None:
        print('regex8')
        return filter_data_size_regex8(toto)
        
    elif cas9_regex.search(toto) != None:
        print('regex9')
        return filter_data_size_regex9(toto)
    
    elif cas10_regex.search(toto) != None:
        print('regex10')
        return filter_data_size_regex10(toto)
        
    elif cas11_regex.search(toto) != None:
        print('regex11')
        return filter_data_size_regex11(toto)
    
    elif cas12_regex.search(toto) != None:
        print('regex12')
        return filter_data_size_regex12(toto)
    
    elif cas13_regex.search(toto) != None:
        print('regex13')
        return filter_data_size_regex13(toto)
        
    elif cas14_regex.search(toto) != None:
        print('regex14')
        return filter_data_size_regex14(toto)
    
    elif cas15_regex.search(toto) != None:
        print('regex15')
        return filter_data_size_regex15(toto)
    
    elif cas16_regex.search(toto) != None:
        print('regex16')
        return filter_data_size_regex16(toto)
        
    elif cas17_regex.search(toto) != None:
        print('regex17')
        height = 0, 
        length = 0
        width = 0

        return height, length, width
    
    elif cas18_regex.search(toto) != None:
        print('regex18')
        return filter_data_size_regex18(toto)
        
    elif cas19_regex.search(toto) != None:
        print('regex19')
        return filter_data_size_regex19(toto)
    elif cas20_regex.search(toto) != None:
        print('regex20')
        return filter_data_size_regex20(toto)
    
    elif cas22_regex.search(toto) != None:
        print('regex22')
        return filter_data_size_regex22(toto)
    
    elif cas23_regex.search(toto) != None:
            print('regex19')
            return filter_data_size_regex23(toto)
        
    elif cas24_regex.search(toto) != None:
            print('regex19')
            return filter_data_size_regex24(toto)

In [70]:
text = 'Inches: 26" W x 26" D x 25.88" H'

In [71]:
regex_pass(text)

regex8
['Inches: 26" W ', ' 26" D ', ' 25.88" H']
width Inches: 26" W 
['Inches:', '26"', 'W', '']
W

width 66.04
lenght  26" D 
['', '26"', 'D', '']

D

lenght 66.04
hight  25.88" H
 25.88" H
['', '25.88"', 'H']

H
hight 65.73519999999999
W: 66.04
L: 66.04
H: 65.73519999999999


(66.04, 66.04, 65.73519999999999)

In [197]:
len(toto_images)

372

In [15]:
col = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/scrapingalrugaibfurniture/sub_categories/model_images.xlsx')

In [16]:
col.columns

Index(['url', 'images1', 'image2'], dtype='object')

In [4]:
df =  pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/scrapingalrugaibfurniture/sub_categories/Rugaid_update_06_08_2021v2.xlsx')

In [205]:
df.shape

(10459, 37)

In [206]:
df['categories'] = ''

In [170]:
df

,sku,is_in_stock,name,price,special_price,short_description,free_colors,description,ts_dimensions_width,ts_dimensions_height,...,swatch_image,thumbnail_image,attribute_set_code,product_websites,out_of_stock_qty,qty,product_online,mgs_brand,estimated_delivery_text,supplier


In [207]:
brand_excel = pd.read_excel('/home/wafistos/Downloads/Rugaib categories and brands.xlsx')

In [214]:
brand_excel.shape

(69, 26)

In [211]:
list_brands_dic = []
for index, row in brand_excel.iterrows():
    list_brands_dic.append([row['Name of category '], row['Num of category '], row['Discription ']])

In [244]:
df.loc[df['short_description'].isnull()]

,sku,is_in_stock,name,price,special_price,short_description,free_colors,description,ts_dimensions_width,ts_dimensions_height,...,thumbnail_image,attribute_set_code,product_websites,out_of_stock_qty,qty,product_online,mgs_brand,estimated_delivery_text,supplier,categories


In [248]:
df_msg = pd.read_excel('/home/wafistos/Downloads/Rugaib categories and brands(1).xlsx')

In [249]:
df_msg.columns

Index(['collection', 'mgs_brand', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')

In [252]:
list_mgs_dic = []
for index, row in df_msg.iterrows():
    list_mgs_dic.append([row['collection'], row['mgs_brand']])

In [256]:
for titi in list_mgs_dic:
    df.loc[df['collection'] == titi[0], 'mgs_brand'] = titi[1] 

In [245]:
df.columns

Index(['sku', 'is_in_stock', 'name', 'price', 'special_price',
       'short_description', 'free_colors', 'description',
       'ts_dimensions_width', 'ts_dimensions_height', 'ts_dimensions_length',
       'collection', 'base_image', 'url_key', 'visibility', 'tax_class_name',
       'manufacturer', 'news_from_date', 'news_to_date', 'additional_images',
       'link_url', 'categories1', 'categories2', 'categories3',
       'allow_backorders', 'estimated_delivery_enable', 'small_image',
       'swatch_image', 'thumbnail_image', 'attribute_set_code',
       'product_websites', 'out_of_stock_qty', 'qty', 'product_online',
       'mgs_brand', 'estimated_delivery_text', 'supplier', 'categories'],
      dtype='object')

In [257]:
df.to_excel('toto.xlsx')

In [233]:
df.loc[df['categories2'] == 'إطارات الصور']

,sku,is_in_stock,name,price,special_price,short_description,free_colors,description,ts_dimensions_width,ts_dimensions_height,...,thumbnail_image,attribute_set_code,product_websites,out_of_stock_qty,qty,product_online,mgs_brand,estimated_delivery_text,supplier,categories
4658,ROG-13651-01,1,DECOR ITEM,110.00,__EMPTY__VALUE__,Material-METAL - MDF - PVC\nColor-WHITE\nWeigh...,WHITE,Material-METAL - MDF - PVC\nColor-WHITE\nWeigh...,__EMPTY__VALUE__,__EMPTY__VALUE__,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,Default,base,-5,0,1,__EMPTY__VALUE__,__EMPTY__VALUE__,ROG,
4659,ROG-A2000159,1,Photo Frame (Set of 2),465.75,__EMPTY__VALUE__,Fond memories don’t have to fade. Capture them...,__EMPTY__VALUE__,Fond memories don’t have to fade. Capture them...,__EMPTY__VALUE__,__EMPTY__VALUE__,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,Default,base,-5,0,1,__EMPTY__VALUE__,__EMPTY__VALUE__,ROG,
4660,ROG-A2080001,1,PHOTO FRAME,115.00,__EMPTY__VALUE__,Master Item Number-\nA2080001\nCarton Contains...,White/Purple,Master Item Number-\nA2080001\nCarton Contains...,__EMPTY__VALUE__,__EMPTY__VALUE__,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,Default,base,-5,0,1,__EMPTY__VALUE__,__EMPTY__VALUE__,ROG,
4661,ROG-15682-02,1,5X5 RESIN ROUND SWIRLS PHOTO FRAME-BLACK,150.00,__EMPTY__VALUE__,12.7 W inch X12.7 H Inch,__EMPTY__VALUE__,12.7 W inch X12.7 H Inch,__EMPTY__VALUE__,__EMPTY__VALUE__,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,Default,base,-5,0,1,__EMPTY__VALUE__,__EMPTY__VALUE__,ROG,
4662,ROG-A2080005,1,PHOTO FRAME (SET OF 2),115.00,__EMPTY__VALUE__,A2080005,__EMPTY__VALUE__,A2080005,__EMPTY__VALUE__,__EMPTY__VALUE__,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,Default,base,-5,0,1,__EMPTY__VALUE__,__EMPTY__VALUE__,ROG,
4663,ROG-18567-UTT,1,Daria Photo Frames-S-3,759.00,__EMPTY__VALUE__,Display those special photos in these sophisti...,__EMPTY__VALUE__,Display those special photos in these sophisti...,__EMPTY__VALUE__,__EMPTY__VALUE__,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,Default,base,-5,0,1,__EMPTY__VALUE__,__EMPTY__VALUE__,ROG,
4664,ROG-18564-UTT,1,Ambrosia Photo Frames-S-3,816.50,__EMPTY__VALUE__,Frames feature o-idized copper sheeting. The o...,__EMPTY__VALUE__,Frames feature o-idized copper sheeting. The o...,__EMPTY__VALUE__,__EMPTY__VALUE__,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,Default,base,-5,0,1,__EMPTY__VALUE__,__EMPTY__VALUE__,ROG,
4665,ROG-14969-02,1,POLYRESIN 5X7- STARFISH PHOTO FRAME-SILVER,170.00,__EMPTY__VALUE__,COLOR-SILVER\nMATERIAL1-POLYRESIN\nTREND-COAST...,SILVER,COLOR-SILVER\nMATERIAL1-POLYRESIN\nTREND-COAST...,__EMPTY__VALUE__,__EMPTY__VALUE__,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,Default,base,-5,0,1,__EMPTY__VALUE__,__EMPTY__VALUE__,ROG,
4666,ROG-13651-02,1,WHITE MARBLE-GOLD PHOTO FRAME17-,109.25,__EMPTY__VALUE__,Material-METAL - MDF - PVC\nColor-WHITE\nWeigh...,WHITE,Material-METAL - MDF - PVC\nColor-WHITE\nWeigh...,__EMPTY__VALUE__,__EMPTY__VALUE__,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,Default,base,-5,0,1,__EMPTY__VALUE__,__EMPTY__VALUE__,ROG,
4667,ROG-A2000193,1,PHOTO FRAME,230.00,__EMPTY__VALUE__,A2000193,__EMPTY__VALUE__,A2000193,__EMPTY__VALUE__,__EMPTY__VALUE__,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,Default,base,-5,0,1,__EMPTY__VALUE__,__EMPTY__VALUE__,ROG,


In [154]:
driver.get('https://www.alrugaibfurniture.com/ar/collections/paintings-pictures/products/painting-70021')
list_images = []
time.sleep(0.5)
images_click = driver.find_elements_by_class_name('product-gallery__thumbnail')


for ele in images_click:
    #ele.click()
    driver.execute_script("arguments[0].click();", ele)
    time.sleep(0.5)
images = driver.find_elements_by_class_name('product-gallery__image')
len(images)
for img in images:
    #list_images.append('https:'+img.get_attribute('data-srcset').split(' ')[0])
#     print(':', img.get_attribute('srcset').split(', ')[0].split('?')[0])
    time.sleep(1)
    try:
        t = 'https:'+img.get_attribute('data-srcset').split(', ')[0].split('?')[0]
    except:
        continue
#     print(img.get_attribute('data-srcset'))
#     soup = BeautifulSoup(img.get_attribute('innerHTML'), 'html.parser')
#     print('toto',soup)
    print(t)
    list_images.append(t)
print('len list images',len(list_images))
try:
    timage = ','.join(list_images[1:])
#         list_input_images2.append(timage)
    print(timage)
except:
    timage = ''
print(timage)

toto_images.append([list_images[0], timage],)

https://cdn.shopify.com/s/files/1/0773/5495/products/70021AR_400x.jpg
len list images 1




In [7]:
list_scrape1 = df['link_url'].to_list()

In [8]:
len(list_scrape1)

10459

In [64]:
list_images = []
time.sleep(2)
images_click = driver.find_elements_by_class_name('product-gallery__thumbnail')


for ele in images_click:
    #ele.click()
    driver.execute_script("arguments[0].click();", ele)
    time.sleep(1)
images = driver.find_elements_by_class_name('product-gallery__image')
len(images)
for img in images:
    #list_images.append('https:'+img.get_attribute('data-srcset').split(' ')[0])
#     print(':', img.get_attribute('srcset').split(', ')[0].split('?')[0])
    time.sleep(1)
    try:
        t = 'https:'+img.get_attribute('data-srcset').split(', ')[0].split('?')[0]
    except:
        continue
#     print(img.get_attribute('data-srcset'))
#     soup = BeautifulSoup(img.get_attribute('innerHTML'), 'html.parser')
#     print('toto',soup)
    print(t)
    list_images.append(t)
print('len list images',len(list_images))
print(list_images[0])

https://cdn.shopify.com/s/files/1/0773/5495/products/20242_400x.jpg
len list images 1
https://cdn.shopify.com/s/files/1/0773/5495/products/20242_400x.jpg


In [61]:
df.to_excel('update1_rugaib.xlsx')

In [58]:
toto_des = driver.find_element_by_class_name('expandable-content').text.split('\n')

NoSuchElementException: Message: Unable to locate element: .expandable-content


In [50]:
driver.get('https://www.alrugaibfurniture.com/ar/collections/rugs/products/fusion-rug-blue-multi-colored')

In [51]:
list_products_other = driver.find_elements_by_class_name('block-swatch__item-text')
len(list_products_other)

3

In [52]:
list_products_other[0].click()

In [114]:
list_des_cat1 = []

In [93]:
link_cat = pd.read_excel('link_all_product_cats-08-07-2021-21-18-36.xlsx')
link_cat.columns

Index(['Unnamed: 0', 'liste', 'cat1', 'cat2', 'cat3'], dtype='object')

In [115]:
for index, row in link_cat.iterrows():
    list_des_cat1.append([row['liste'], row['cat1'], row['cat2'], row['cat3'] ], )


In [121]:
all_products = pd.read_excel('Rugaib_update_08_07_2021_v1.xlsx')

In [123]:
all_products.columns

Index(['Unnamed: 0', 'sku', 'is_in_stock', 'name', 'price', 'special_price',
       'short_description', 'free_colors', 'description', 'Width', 'Height',
       'Length', 'mgs_brand', 'base_image', 'url_key', 'visibility',
       'tax_class_name', 'manufacturer', 'news_from_date', 'news_to_date',
       'additional_images ', 'link_url', 'categories1', 'categories2',
       'categories3', 'Unnamed: 0.1', 'raw_materials', 'small_image',
       'swatch_image', 'thumbnail_image', 'attribute_set_code',
       'product_websites', 'allow_backorders', 'estimated_delivery_enable'],
      dtype='object')

In [147]:
all_toto[['url1', 'url2', 'url3', 'url4', 'url5', 'url6', 'url7', 'url8']] =  all_toto['link_url'].str.split('/', expand=True)

In [148]:
all_toto[['url', 'toto']] = all_toto['url8'].str.split('?', expand=True)

In [151]:
all_toto['categories3']

0       الخزائن الجانبية
1          طاولات الطعام
2           كراسي الطعام
3            طاولات مكتب
4          خزائن وصناديق
              ...       
2168             المرايا
2169          سرير اطفال
2170       طاولات الخدمة
2171      ديكورات الحائط
2172           كنب طاولة
Name: categories3, Length: 2173, dtype: object

In [150]:
for cat in list_des_cat1:
    #print(cat[1], 'toto', cat[0])
    all_toto.loc[all_products['url'] == cat[0] , 'categories3'] = cat[3]

In [140]:
for cat in list_des_cat1:
    #print(cat[1], 'toto', cat[0])
    all_products.loc[all_products['url'] == cat[0] , 'categories1'] = cat[1]

In [142]:
for cat in list_des_cat1:
    #print(cat[1], 'toto', cat[0])
    all_products.loc[all_products['url'] == cat[0] , 'categories2'] = cat[2]

In [141]:
all_products['categories3']

0         الخزائن الجانبية
1            طاولات الطعام
2             كراسي الطعام
3              طاولات مكتب
4            خزائن وصناديق
               ...        
9575             كنب زاوية
9576       كرسي طاولة خدمة
9577       كرسي طاولة خدمة
9578    طاولات سرير جانبية
9579        مجموعات الجدول
Name: categories3, Length: 9580, dtype: object

# Traitement 

In [21]:
df.to_excel('toto.xlsx')

In [75]:

df[['price2', 'price1']] = df['price'].str.split("-",expand=True,)


In [77]:
df['price']  = df['price2']

In [16]:
df['dimension'] = df['dimension'].str.replace('cm', '')

In [17]:
df['dimension'] = df['dimension'].str.replace('Cm', '')

In [18]:
df['dimension'] = df['dimension'].str.replace('×', 'x')

In [19]:
df[['size', 'size1']] = df['dimension'].str.split(",",expand=True,)

In [22]:

df[['ts_dimensions_width','ts_dimensions_height', 'ts_dimensions_length']] = df['size'].str.split("x",expand=True,)


In [76]:
df.to_excel('toto.xlsx')

In [56]:
df.loc[df['ts_dimensions_length'].isnull(), 'ts_dimensions_length'] = '__EMPTY__VALUE__'

In [57]:
df.loc[(df['ts_dimensions_width'] == 0) | (df['ts_dimensions_width'].isnull()), 'ts_dimensions_width'] = '__EMPTY__VALUE__'

In [58]:
df.loc[(df['ts_dimensions_length'] == 0) | (df['ts_dimensions_length'].isnull()), 'ts_dimensions_length'] = '__EMPTY__VALUE__'

In [59]:
df.loc[(df['ts_dimensions_height'] == 0) | (df['ts_dimensions_height'].isnull()), 'ts_dimensions_height'] = '__EMPTY__VALUE__'

In [23]:
df['supplier'] = 'HOM'

In [31]:
df.loc[df['is_in_stock']!='1',  'is_in_stock' ] = '0'

In [32]:
df['allow_backorders'] = df['is_in_stock']
df['estimated_delivery_enable'] = 'Static, Text'

In [33]:
df['small_image'] =  df['base_image']
df['swatch_image'] =  df['base_image']
df['thumbnail_image'] =  df['base_image']
df['attribute_set_code'] = 'Default'
df['product_websites'] = 'base'

In [34]:
df['out_of_stock_qty'] = -5
df['qty'] = 0
df['product_online'] = df['is_in_stock']

In [35]:
df.loc[df['product_online']== '0', 'product_online'] = '2'

# Traitement fonctions

In [36]:
def toto_clean(name):
    df[name] = df[name].str.replace('•', '-')
    df[name] = df[name].str.replace(':', '-')
    df[name] = df[name].str.replace('/', '-')
    df[name] = df[name].str.replace('"', '-')
    df[name] = df[name].str.replace('%', '-')
    df[name] = df[name].str.replace('&', '-')
    df[name] = df[name].str.replace('?', '-', regex=False)
    df[name] = df[name].str.replace('(', '-', regex=False)
    df[name] = df[name].str.replace(')', '-', regex=False)
    df[name] = df[name].str.replace('{', '-', regex=False)
    df[name] = df[name].str.replace('}', '-', regex=False)
    df[name] = df[name].str.replace("'", '-', regex=False)
    df[name] = df[name].str.replace('*', 'X', regex=False)
    df[name] = df[name].str.replace('#', '-')
    df[name] = df[name].str.replace('”', '-')
    df[name] = df[name].str.replace('!', '-')
    df[name] = df[name].str.replace('”', '-')
    df[name] = df[name].str.replace('=', '-')
    df[name] = df[name].str.replace('+', '-', regex=False)
    df[name] = df[name].str.replace('،', '-', regex=False)
    df[name] = df[name].str.replace('عرض أقل', '', regex=False)
    df[name] = df[name].str.replace('،', '-', regex=False)
    
        
    
def ampty_value(name):
    df.loc[df[name].isnull(), name] = '__EMPTY__VALUE__'
    
    
def price_num(name):
    try:
        df[name] = df[name].str.replace('-', '')
        df[name] = df[name].str.replace(',', '')
    except:
        pass
    df.loc[df[name] == '__EMPTY__VALUE__', name] = 0
    
    df[name] = pd.to_numeric(df[name])
    df.loc[df[name] == 0, name] = '__EMPTY__VALUE__'

In [37]:
def clean_punct(name):
    for c in string.punctuation:
        df[name] = df[name].str.replace(c, '-', regex=False)

In [38]:
toto_clean('name')

In [39]:
toto_clean('description')

In [40]:
toto_clean('name')

In [41]:
df.to_excel('test.xlsx')

In [42]:
df['url_key'] = df['sku'] + '-' + df['name']

In [43]:
df['name'] = df['name'].str.replace(', ', '-')
df['name'] = df['name'].str.replace('/', '-')
df['name'] = df['name'].str.replace('"', '-')
df['name'] = df['name'].str.replace('%', '-')
df['name'] = df['name'].str.replace('&', '-')
df['name'] = df['name'].str.replace('+', '-', regex=False)

In [44]:
df['estimated_delivery_enable'] = 'Static Text'

In [45]:
df['categories'] = ''

In [46]:
df['price'] = df['price'].str.replace(',', '')

In [47]:
df['special_price'] = df['special_price'].str.replace(',', '')

KeyError: 'special_price'

In [54]:
df['categories1'] = df['cat1']
df['categories2'] = df['cat2']
df['categories3'] = df['cat3']
df['link_url'] = df['url']

In [121]:
df['price'] = pd.to_numeric(df['price'])

In [122]:
df['special_price'] = pd.to_numeric(df['special_price'])

In [123]:
df['collection'] = df['collection'].str.title()

In [124]:
df['supplier'] = 'ROG'

# Categories add

In [464]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/scrapingalrugaibfurniture/sub_categories/Rugaib_update_27_08_2021_v3.xlsx')

In [481]:
list_df = pd.read_excel('/home/wafistos/Downloads/Missing description.xlsx')

In [468]:
df_toto = df[['sku', 'categories1', 'categories2', 'categories3', 'categories'] ].copy()

In [95]:
cats = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/scrapingalrugaibfurniture/sub_categories/categories/Rugaib categories .xlsx')

In [97]:
cats.columns

Index(['Name of category ', 'Num of category ', 'Discription ', 'Unnamed: 3',
       'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25'],
      dtype='object')

In [99]:
list_cats = []
for index, row in cats.iterrows():
    list_cats.append([ row['Name of category '], row['Num of category '], row['Discription ']])

In [100]:
list_cats

[['  غرفة المعيشة ',
  '43441,42959,113',
  'لكي نضمن راحتكم ولتوفير كافة مستلزمات غرفة المعيشة مع اختصار الوقت اللازم للبحث عنها في أماكن متفرقة وتجميع قطع الأثاث قام مصممو وفنانو بيتونيا بتحضير أفخم غرف المعيشة المميزة لكم والمتناسبة مع تشكيلة واسعة من تصاميم المنازل العصرية. القطع منتقاة بعناية وكلها مصنوعة من مواد عالية الجودة, مقاومة ومتينة.'],
 ['حامل شمعة',
  '43816,43,113',
  '        مصمم بأناقة لينمزج جيداً مع المكونات الداخلية للغرفة. مصنوعة من مواد فائقة الجودة لتعزيز المتانة وزجاج ذات ظلال متداخلة مزدوجة لتوفر كلاً من اللون والملمس'],
 ['شموع',
  '43816,43506,113',
  'شموع فاخرة تضفي على أمسياتك لمية من الدفء والشاعرية وتمنحك ساعات من الاسترخاء على ضوءها الخافت بعد ساعات من العمل الطويل الشاق. مثالية لعشاء رومانسي على ضوء الشموع أو لإضاءة خافتة في المناسبات الخاصة تخلق جوا عاما مفعما بالدفء والجمال.'],
 ['الساعات',
  '42,43504,51,113',
  'إذا كنت تريد أن تزيد غرفة المعيشة الخاصة بك توهجا وتألق ، فهذا التصميم سيعكس اختيارك ساعة حائط بيرام ساعة'],
 ['وسائد',
  '43509,43213,1

In [102]:
list_cats[0][1]

'43441,42959,113'

In [105]:
for ll in list_cats:
    df.loc[df['categories3']== ll[0], 'categories'] = ll[1]

In [108]:
df.loc[df['categories']=='', 'categories2']

11226    ألعاب
11227    ألعاب
11228    ألعاب
11229    ألعاب
11230    ألعاب
11231    ألعاب
Name: categories2, dtype: object

In [492]:
list_df.loc[list_df['cat']=='']

,sku,name,description,short_description,cat
0,DAS-15414,بلمباجو جولد,NaN,NaN,
1,DAS-82640,كرسى مكتب ظهر عالى نوبليزا,NaN,NaN,
2,DAS-19337,انجى,NaN,NaN,
3,DAS-129452,طقم فناجين قهوة 6 قطع فيناتينو,NaN,NaN,
4,DAS-107469,بورتافاليس,NaN,NaN,
...,...,...,...,...,...
3453,ROG-CVTOP2602,CVTOP2602 لوحة,NaN,NaN,
3454,ROG-T341-9,Ezmonei Lift Top طاولة قهوة,NaN,NaN,
3455,ROG-cas-coctab-001,cas-coctab-001,NaN,NaN,
3456,ROG-D803-01A,Dining Chairكرسي,NaN,NaN,


In [491]:
list_df.columns

Index(['sku', 'name', 'description', 'short_description', 'cat'], dtype='object')

In [494]:
df.loc[df['sku']=='ROG-CVTOP2602']

,sku number only,sku,store_view_code,attribute_set_code,product_websites,name,description,short_description,link_url,categories1,...,thumbnail_image,additional_images,product_online,qty,out_of_stock_qty,allow_backorders,is_in_stock,estimated_delivery_enable,estimated_delivery_text,supplier
1117,CVTOP2602,ROG-CVTOP2602,NaN,Default,base,CVTOP2602,47 X 47-- 100- HAND PAINTED CANVAS,47 X 47-- 100- HAND PAINTED CANVAS,https://www.alrugaibfurniture.com/ar/collectio...,اكسسوارات,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,NaN,1,0,-5,1,1,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,ROG


In [526]:
list_df.to_excel('updatetoto2.xlsx')

In [523]:
list_df.loc[list_df['cat']=='']

,sku,name,description,short_description,cat
2438,ROG-100087074,Dallaty Vacuum Flask 1 Pieces Pot Either 1L,NaN,NaN,
2961,ROG-100072575,Dallety Plastic Vacuum Flask 1 L,NaN,NaN,
3345,ROG-14156B-UTT,مرآة جدارية بيرغامو,NaN,NaN,


In [517]:
df1 = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/scrapingalrugaibfurniture/sub_categories/Rugaib_update_20_08_2021v6.xlsx')

In [518]:
df1.columns

Index(['Unnamed: 0', 'sku', 'store_view_code', 'attribute_set_code',
       'product_websites', 'name', 'description', 'short_description',
       'link_url', 'categories1', 'categories2', 'categories3', 'categories',
       'free_colors', 'ts_dimensions_width', 'ts_dimensions_height',
       'ts_dimensions_length', 'mgs_brand', 'url_key', 'cost', 'price',
       'special_price', 'visibility', 'product_type', 'tax_class_name',
       'manufacturer', 'news_from_date', 'news_to_date', 'base_image',
       'small_image', 'swatch_image', 'thumbnail_image', 'additional_images',
       'product_online', 'qty', 'out_of_stock_qty', 'allow_backorders',
       'is_in_stock', 'estimated_delivery_enable', 'estimated_delivery_text',
       'supplier'],
      dtype='object')

In [519]:
toto_list = []
for index, row in df1.iterrows():
    toto_list.append([ row['sku'], row['categories']])
    

In [522]:
for tt in toto_list:
    list_df.loc[list_df['sku']==tt[0], 'cat'] = tt[1]

In [521]:
toto

,sku,name,description,short_description,cat
1366,ROG-B777-46,وحده الادراج,NaN,NaN,"43507,6711,113"
1400,ROG-15852-03,"116""h textured olpe vase, black",NaN,NaN,"42,43741,113"
1491,ROG-B777-36,مرايا,NaN,NaN,"42,14489,113"
1648,ROG-B777-31,Baldridge Home Office Desk Chair,NaN,NaN,"43443,43278,113"
1790,ROG-B777-93,كومدينو -B777-93,NaN,NaN,"43443,43727,113"
1808,ROG-28260-1-UTT,Higgins Floor Lamp,NaN,NaN,"6723,43668,113"
2205,ROG-D120-130,Tall UPH Swivel كرسي مرتفع(2/CN),NaN,NaN,"43440,26,113"
2376,ROG-H741-46,H741-46/49 Luxenford,NaN,NaN,"23,43649,43734,113"
2386,ROG-B267-36,مرايا b267-31,NaN,NaN,"43443,43278,113"
2438,ROG-100087074,Dallaty Vacuum Flask 1 Pieces Pot Either 1L,NaN,NaN,


In [499]:
midas_scrap = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/Scraping_Midas/update midas/Midas_update_27_08_2021v3.xlsx')

In [500]:
list_cats = []
for index, row in midas_scrap.iterrows():
    list_cats.append([ row['sku'], row['categories']])

In [501]:
for ll in list_cats:
    list_df.loc[list_df['sku']== ll[0], 'cat'] = ll[1]

In [507]:
list_cats

[['  غرفة المعيشة ',
  '43441,42959,113',
  'لكي نضمن راحتكم ولتوفير كافة مستلزمات غرفة المعيشة مع اختصار الوقت اللازم للبحث عنها في أماكن متفرقة وتجميع قطع الأثاث قام مصممو وفنانو بيتونيا بتحضير أفخم غرف المعيشة المميزة لكم والمتناسبة مع تشكيلة واسعة من تصاميم المنازل العصرية. القطع منتقاة بعناية وكلها مصنوعة من مواد عالية الجودة, مقاومة ومتينة.'],
 ['حامل شمعة',
  '43816,43,113',
  '        مصمم بأناقة لينمزج جيداً مع المكونات الداخلية للغرفة. مصنوعة من مواد فائقة الجودة لتعزيز المتانة وزجاج ذات ظلال متداخلة مزدوجة لتوفر كلاً من اللون والملمس'],
 ['شموع',
  '43816,43506,113',
  'شموع فاخرة تضفي على أمسياتك لمية من الدفء والشاعرية وتمنحك ساعات من الاسترخاء على ضوءها الخافت بعد ساعات من العمل الطويل الشاق. مثالية لعشاء رومانسي على ضوء الشموع أو لإضاءة خافتة في المناسبات الخاصة تخلق جوا عاما مفعما بالدفء والجمال.'],
 ['الساعات',
  '42,43504,51,113',
  'إذا كنت تريد أن تزيد غرفة المعيشة الخاصة بك توهجا وتألق ، فهذا التصميم سيعكس اختيارك ساعة حائط بيرام ساعة'],
 ['وسائد',
  '43509,43213,1

In [525]:
for ll in list_cats:
    
    list_df.loc[list_df['cat'] == ll[1] , 'short_description'] = ll[2]

In [ ]:
list_df

In [504]:
cats = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/scrapingalrugaibfurniture/sub_categories/categories/Rugaib categories .xlsx')

In [505]:
list_cats = []
for index, row in cats.iterrows():
    list_cats.append([row['Name of category '], row['Num of category '], row['Discription ']] )
    

In [182]:
df.to_excel("Rugaib_update_27_08_2021v3.xlsx")

In [129]:
df.drop(df.loc[df['categories'] == 'Do not scrap it '].index, inplace=True)


In [181]:
df.loc[df['categories'] == 'Do not scrap it ']

,sku,store_view_code,attribute_set_code,product_websites,name,description,short_description,link_url,categories1,categories2,...,thumbnail_image,additional_images,product_online,qty,out_of_stock_qty,allow_backorders,is_in_stock,estimated_delivery_enable,estimated_delivery_text,supplier


In [362]:
list_cats[12][0]

'إطارات الصور'

In [228]:
df = None 

In [229]:
df = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/scrapingalrugaibfurniture/sub_categories/Rugaib_update_20_08_2021v6.xlsx')

In [435]:
for ll in list_cats:
    
    df.loc[df['categories3'] == ll[0]  , 'categories'] = ll[1]

In [139]:
for ll in list_cats:
    
    df.loc[(df['categories'] == ll[1]) & (df['description'].str.len() <= 20) , 'description'] = ll[2]

In [437]:
for ll in list_cats:
    
    df.loc[(df['categories'] == ll[1]) & (df['short_description'].str.len() <= 20) , 'short_description'] = ll[2]

In [441]:
 df.loc[ (df['categories'].isnull()) | (df['categories'] == '') , 'categories2']

9245    ألعاب
9246    ألعاب
9247    ألعاب
9248    ألعاب
9249    ألعاب
9250    ألعاب
9251    ألعاب
Name: categories2, dtype: object

In [140]:
df.loc[df['description'].str.len() <= 20]

,sku number only,sku,store_view_code,attribute_set_code,product_websites,name,description,short_description,link_url,categories1,...,thumbnail_image,additional_images,product_online,qty,out_of_stock_qty,allow_backorders,is_in_stock,estimated_delivery_enable,estimated_delivery_text,supplier


In [ ]:


Try this:

df.drop(df.loc[df.time< df.time.shift()].index, inplace=True)



In [214]:
df.shape

(12042, 40)

In [145]:
df.to_excel('Rugaib_update_27_08_2021v1.xlsx')

In [216]:
aa

,sku,store_view_code,attribute_set_code,product_websites,name,description,short_description,link_url,categories1,categories2,...,thumbnail_image,additional_images,product_online,qty,out_of_stock_qty,allow_backorders,is_in_stock,estimated_delivery_enable,estimated_delivery_text,supplier
46,ROG-100091152,NaN,Default,base,Glass Ribbed Candle Holder Solid Silver Finish,10X27 CM,10X27 CM,https://www.alrugaibfurniture.com/ar/collectio...,اكسسوارات,حامل شمعة,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,NaN,1,0,-5,1,1,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,ROG
122,ROG-100090475,NaN,Default,base,Candle Holder Arabesque Design Gold and Wood Dia,23X32 CM,23X32 CM,https://www.alrugaibfurniture.com/ar/collectio...,اكسسوارات,حامل شمعة,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,NaN,1,0,-5,1,1,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,ROG
123,ROG-100090476,NaN,Default,base,Candle Holder Arabesque Design Gold and Wood Dia,23X28 CM,23X28 CM,https://www.alrugaibfurniture.com/ar/collectio...,اكسسوارات,حامل شمعة,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,NaN,1,0,-5,1,1,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,ROG
178,ROG-100090481,NaN,Default,base,Metal Candle Holder Gold Medium,13X18 CM,13X18 CM,https://www.alrugaibfurniture.com/ar/collectio...,اكسسوارات,حامل شمعة,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,NaN,1,0,-5,1,1,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,ROG
180,ROG-100090480,NaN,Default,base,Metal Candle Holder Gold Large,16X25 CM,16X25 CM,https://www.alrugaibfurniture.com/ar/collectio...,اكسسوارات,حامل شمعة,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,NaN,1,0,-5,1,1,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,ROG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11983,ROG-D394-024,NaN,Default,base,UPHOLSTERED STOOL,44x44x60,44x44x60,https://www.alrugaibfurniture.com/ar/collectio...,أثاث,غرف الطعام,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,NaN,1,0,-5,1,1,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,ROG
12031,ROG-D813-124,NaN,Default,base,WYNDAHL COUNTER HEIGHT BAR STOOL,50x54x105,50x54x105,https://www.alrugaibfurniture.com/ar/collectio...,أثاث,غرف الطعام,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,NaN,1,0,-5,1,1,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,ROG
12032,ROG-D743-124,NaN,Default,base,UPHOLSTERED BARSTOOL,50x60x106,50x60x106,https://www.alrugaibfurniture.com/ar/collectio...,أثاث,غرف الطعام,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,NaN,1,0,-5,1,1,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,ROG
12033,ROG-D712-124,NaN,Default,base,STOOL,43x38x61,43x38x61,https://www.alrugaibfurniture.com/ar/collectio...,أثاث,غرف الطعام,...,https://cdn.shopify.com/s/files/1/0773/5495/pr...,NaN,1,0,-5,1,1,Static Text,الوقت المقدر لشحن هذا المنتج أسبوعين,ROG


In [213]:
df.loc[df['categories2'] == 'اثاث خارجي', 'categories'] = '14490,113'

8211    14490,113
8212    14490,113
8213    14490,113
8214    14490,113
8215    14490,113
          ...    
8276    14490,113
8277    14490,113
8278    14490,113
8279    14490,113
8280    14490,113
Name: categories, Length: 70, dtype: object

In [460]:


df.drop(df.loc[df['categories'] == 'Do not scrap it '].index, inplace=True)



In [210]:
df.to_excel('Rugaib_update_20_08_2021v6.xlsx')

In [116]:
df.loc[df['short_description'].isnull()]

,sku,is_in_stock,name,price,special_price,short_description,free_colors,description,ts_dimensions_width,ts_dimensions_height,...,swatch_image,thumbnail_image,attribute_set_code,product_websites,out_of_stock_qty,qty,product_online,estimated_delivery_text,mgs_brand,categories


# Collections add 

In [109]:
cols = pd.read_excel('/home/wafistos/Documents/Projects/scaping_wafi/scrapingalrugaibfurniture/sub_categories/categories/Rugaib brands.xlsx')

In [443]:
cols.columns

Index(['collection', 'mgs_brand', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13',
       'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17',
       'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21',
       'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')

In [110]:
list_cols = []
for index, row in cols.iterrows():
    list_cols.append([ row['collection'], row['mgs_brand']])

In [111]:
for col in list_cols:
    df.loc[df['collection'] == col[0], 'mgs_brand'] = col[1]

In [264]:
df['categories3'] = ''

In [114]:
df['mgs_brand']

0        Ashley
1        Ashley
2           NaN
3        Ashley
4        Ashley
          ...  
12215       NaN
12216       NaN
12217       NaN
12218       NaN
12219       NaN
Name: mgs_brand, Length: 12113, dtype: object

In [61]:
now = datetime.now()
two_month = datetime.now() + timedelta(days=60)
two_month = two_month.strftime("%m/%d/%Y")
today = datetime.today().strftime("%m/%d/%Y")
df['news_from_date'] = today
df['news_to_date'] = two_month

In [63]:

df.columns

Index(['sku', 'name', 'price', 'free_colors', 'raw_material', 'description',
       'barcode', 'type_', 'dimension', 'mgs_brand', 'base_image',
       'additionnnal_images', 'tody', 'two_month', 'url', 'manufacturer',
       'delevry_days', 'is_in_stock', 'cat1', 'cat2', 'cat3', 'size', 'size1',
       'ts_dimensions_width', 'ts_dimensions_height', 'ts_dimensions_length',
       'supplier', 'allow_backorders', 'estimated_delivery_enable',
       'small_image', 'swatch_image', 'thumbnail_image', 'attribute_set_code',
       'product_websites', 'out_of_stock_qty', 'qty', 'product_online',
       'url_key', 'categories', 'sku number only', 'product_type',
       'store_view_code', 'cost', 'categories1', 'categories2', 'categories3',
       'link_url', 'estimated_delivery_text', 'special_price',
       'short_description', 'tax_class_name', 'news_from_date',
       'news_to_date'],
      dtype='object')

In [64]:
df['additional_images'] = df['additionnnal_images']

In [58]:
df['tax_class_name'] = 'Taxable Goods'

In [65]:
df['visibility'] = 'Catalog, Search'

In [57]:
df['short_description'] = ''

In [79]:
df['cost'] = df['price']

In [50]:
df['store_view_code'] = ''

In [56]:
df['special_price'] ='__EMPTY__VALUE__'

In [48]:
df['sku number only'] = df['sku'].str.replace('HOM-', '')

In [49]:
df['product_type'] = 'simple'

In [121]:
df['price'] = df['price'] * 1.1

In [124]:
df['special_price'] = df['special_price'] * 1.1

In [55]:
df['estimated_delivery_text'] = ''

In [70]:
df = df[[ 'sku number only',
        'sku', 
         'store_view_code',
         'attribute_set_code',
         'product_websites',
         'name',   
         'description', 
         'short_description', 
         'link_url', 
         'categories1', 
         'categories2', 
         'categories3', 
         'categories', 
         'free_colors', 
         'ts_dimensions_width', 
         'ts_dimensions_height', 
         'ts_dimensions_length',
         'mgs_brand',
         'url_key',
         'cost',  
         'price', 
         'special_price', 
         'visibility',
         'product_type',
         'tax_class_name', 
         'manufacturer',
         'news_from_date', 
         'news_to_date', 
         'base_image', 
         'small_image', 
         'swatch_image' , 
         'thumbnail_image', 
         'additional_images', 
         'product_online', 
         'qty', 
         'out_of_stock_qty', 
         'allow_backorders', 
         'is_in_stock', 
         'estimated_delivery_enable',
         'estimated_delivery_text',
         'supplier',   
        ]]

In [ ]:
df['']

In [80]:
df.to_excel('HOM.xlsx', index=False)

In [73]:
df['price'] = df['price'].str.replace('\n', '')

In [137]:
df = pd.read_excel('Rugaib_update_03_09_2021_v1.xlsx')

In [69]:
df['price'] = df['price'].str.strip()

<ipython-input-69-32dc630862f2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].str.strip()


In [398]:
toto_clean('url_key')

In [457]:
df['url_key'] = df['url_key'].str.replace(' ', '-')

In [458]:
df['url_key'] = df['url_key'].str.replace('  ', '-')

In [ ]:
df.drop(df.loc[df['sku'] == 'NONE'].index, inplace=True)

In [6]:
df = pd.read_excel('/home/wafistos/Downloads/homzmart.xlsx')

In [ ]:
aaa